In [1]:
# python script of M4_weekly and DeepAR

# standard imports
import numpy as np
import pandas as pd

# json
import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator


# model imports
# from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

# seeds
mx.random.seed(42)
np.random.seed(42)

if __name__ == "__main__":

    dataset = get_dataset("m4_hourly", regenerate=False)

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=1,      # default: 100
        num_batches_per_epoch=50,      #default: 50
        learning_rate=1e-3,
        # hybridize=False,
    )

    # cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        # context_length=168,
        # cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )

    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )

pprint(agg_metrics)

INFO:root:Using CPU
INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_hourly.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 13463
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:13<00:00,  3.84it/s, avg_epoch_loss=5.07]
INFO:root:Epoch[0] Elapsed time 13.061 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.065082
INFO:root:Loading parameters from best epoch (0)
INFO:root:Final loss: 5.065082225799561 (occurred at epoch 0)
INFO:root:End model training
Running evaluation: 100%|██████████| 414/414 [01:43<00:00,  3.99it/s]


{'Coverage[0.1]': 0.07996175523349439,
 'Coverage[0.2]': 0.10165056360708537,
 'Coverage[0.3]': 0.1193136070853462,
 'Coverage[0.4]': 0.13909017713365537,
 'Coverage[0.5]': 0.16435185185185172,
 'Coverage[0.6]': 0.20309983896940426,
 'Coverage[0.7]': 0.28658413848631237,
 'Coverage[0.8]': 0.41817632850241554,
 'Coverage[0.9]': 0.574728260869565,
 'MAE_Coverage': 0.26811594202898553,
 'MASE': 4.23782632849611,
 'MSE': 102437142.28404614,
 'MSIS': 49.13415579244133,
 'ND': 0.22644738127027098,
 'NRMSE': 1.3817569234516849,
 'QuantileLoss[0.1]': 10956273.862854863,
 'QuantileLoss[0.2]': 18399496.65324364,
 'QuantileLoss[0.3]': 24530474.890988257,
 'QuantileLoss[0.4]': 29382854.531607054,
 'QuantileLoss[0.5]': 32961422.988054276,
 'QuantileLoss[0.6]': 35557852.25317364,
 'QuantileLoss[0.7]': 35731462.47803888,
 'QuantileLoss[0.8]': 32631821.722134784,
 'QuantileLoss[0.9]': 22708216.433294013,
 'RMSE': 10121.12356826287,
 'abs_error': 32961423.482808113,
 'abs_target_mean': 7324.82204104314